# Build Docker Image

In [2]:
%%writefile dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        cmake \
        git \
        curl \
        ca-certificates && \
    rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.7
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy scikit-learn && \
    /opt/conda/bin/conda clean -ya && \
    ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh && \
    echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && \ 
    echo "conda activate py$PYTHON_VERSION" >> ~/.bashrc

ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
        
RUN /opt/conda/bin/conda install --name py$PYTHON_VERSION -c pytorch pytorch=1.1.0 torchvision=0.3.0 cudatoolkit=9.0 \
    matplotlib=3.1.0 Cython=0.29.10
    
RUN pip install --upgrade pip && \
    pip install --upgrade azureml-sdk[notebooks]==1.0.41 && \
    /opt/conda/bin/conda clean -yt

RUN git clone https://github.com/cocodataset/cocoapi.git && \
    cd cocoapi/PythonAPI && make && cd -

Overwriting dockerfile


In [6]:
docker_login = 'fboylu'
image_repo = 'torchdet'

In [8]:
image_name = docker_login + '/' + image_repo
image_name

'fboylu/torchdet'

In [9]:
!docker build -t $image_name .

Sending build context to Docker daemon  251.4MB
Step 1/9 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 65dee97b9662
Step 2/9 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         cmake         git         curl         ca-certificates &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 2515c1b02bdd
Step 3/9 : ENV PYTHON_VERSION=3.7
 ---> Using cache
 ---> 5cc9f24e56de
Step 4/9 : RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  &&     chmod +x ~/miniconda.sh &&     ~/miniconda.sh -b -p /opt/conda &&     rm ~/miniconda.sh &&     /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy scikit-learn &&     /opt/conda/bin/conda clean -ya &&     ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh &&     echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc &&     echo "conda activate py$PYTHON_VERSION" >> ~/.bashrc
 ---> Using cache
 

```
nvidia-docker run -it -d --shm-size=1g --name torch torchdet
docker cp torchdetect torch:/torchdetect
docker cp /datadrive/torchvisionOD/BuildData/ torch:/torchdetect/BuildData
docker attach torch
python /torchdetect/train_sub.py --data_path /torchdetect/BuildData/ --workers 8 --epochs 5 --anchor_sizes 16 32 64 128 256 512 --anchor_aspect_ratios 0.25 0.5 1.0 2.0 --rpn_nms_thresh 0.5 --box_nms_thresh 0.3 --box_score_thresh 0.10
```

In [10]:
!docker push $image_name

The push refers to repository [docker.io/fboylu/torchdet]

82a0dce1: Preparing 
aec1a673: Preparing 
46f5794a: Preparing 
009b80ee: Preparing 
68871e46: Preparing 
eb3abf5c: Preparing 
7198b4a5: Preparing 
b1166576: Preparing 
61ff09c5: Preparing 
208b2662: Preparing 
a9223884: Preparing 
7198b4a5: Waiting g 
2298c779: Preparing 
eb3abf5c: Waiting g 


6f5794a: Pushing  863.8MB/2.975GBPushing  23.57MB/2.975GBPushing  42.89MB/663.7MBPushing  386.8MB/1.287GBPushing  658.4MB/663.7MBPushing  696.5MB/2.975GB

6f5794a: Pushing  2.215GB/2.975GBPushing  1.215GB/2.975GB

6f5794a: Pushed   3.003GB/2.975GBlatest: digest: sha256:763ab1043957413495b0fd25eba9affdd59b8d6bee4eb83ffd122d941bf2a9bc size: 3482
